In [61]:
from datetime import date, timedelta
import market
from market import get_ohlc, get_stock_list, strong_stocks
import numpy as np, pandas as pd
import ipywidgets as widgets
from pykrx import stock as krx
import importlib
importlib.reload(market)

<module 'market' from 'D:\\work\\python\\market_analysis\\market.py'>

In [2]:
ks_rate, df_ks = strong_stocks('kospi')
kq_rate, df_kq = strong_stocks('kosdaq')

100%|██████████████████████████████████████████████████████████████████████████████| 1746/1746 [00:23<00:00, 74.03it/s]


In [88]:
from mplchart.chart import Chart
from mplchart.primitives import Candlesticks, Volume
from mplchart.indicators import ROC, SMA, EMA, RSI, MACD

def next_stock(button):
    if market_sel.value == '코스피':
        if not next_stock.ks_initial:
            if button == btn_next_chart:
                next_stock.ks_cnt += 1
                next_stock.ks_cnt %= len(df_ks)
            else:
                next_stock.ks_cnt -= 1
                next_stock.ks_cnt %= len(df_ks)
        else:
            next_stock.ks_initial = False
        plot_chart(df_ks.iloc[next_stock.ks_cnt, :])
    else:
        if not next_stock.kq_initial:
            if button == btn_next_chart:
                next_stock.kq_cnt += 1
                next_stock.kq_cnt %= len(df_kq)
            else:
                next_stock.kq_cnt -= 1
                next_stock.kq_cnt %= len(df_kq)
        else:
            next_stock.kq_initial = False
        plot_chart(df_kq.iloc[next_stock.kq_cnt, :])
next_stock.ks_cnt = 0
next_stock.kq_cnt = 0
next_stock.ks_initial = True
next_stock.kq_initial = True


indicators = [
    Candlesticks(), SMA(50), SMA(200), Volume(),
    RSI(),
    MACD(),
]

def plot_chart(strong):
    chart_out.clear_output()
    with chart_out:
        code = strong['code']
        chart = Chart(title=krx.get_market_ticker_name(code), max_bars=480)        
        prices = get_ohlc(code=code).set_index('Date')[['Open', 'High', 'Low', 'Close', 'Volume']]
        print(prices)
        # chart.plot(prices, indicators)        

btn_next_chart = widgets.Button(
    description='Next',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='caret-right' # (FontAwesome names without the `fa-` prefix)
)
btn_next_chart.on_click(next_stock)

btn_prev_chart = widgets.Button(
    description='Prev',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='caret-left' # (FontAwesome names without the `fa-` prefix)
)
btn_prev_chart.on_click(next_stock)

chart_out = widgets.Output(layout={'border': '1px solid black'})

In [89]:
market_sel = widgets.RadioButtons(
    options=['코스피', '코스닥'],
    description='Market Selection',
    disabled=False
)

In [90]:
ctrl_wg = widgets.HBox([btn_prev_chart, btn_next_chart, market_sel])
ui = widgets.VBox([chart_out, ctrl_wg])

In [91]:
display(ui)